In [ ]:
!pip install aiohttp nest_asyncio
!pip install pyngrok

In [ ]:
import psutil
import os

def kill_port(port):
    for proc in psutil.process_iter(['pid', 'name']):
        for conn in proc.connections(kind='inet'):
            if conn.laddr.port == port:
                os.kill(proc.info['pid'], 9)  # Beende den Prozess auf dem angegebenen Port
                print(f"Prozess {proc.info['pid']} auf Port {port} wurde beendet.")

# Port 8081 freigeben, falls er belegt ist
kill_port(8081)

In [ ]:
# start_server.py
import asyncio
from prozessor_utils.main_server import main
from prozessor_utils.ngrok_utils import start_ngrok

async def start_services():
    await main()
    authtoken = "2ltJwdZ4ItmsDom35mIM3kz7wCT_6yKrmv2HpAMhZemSV2iwa"
    await start_ngrok(authtoken, 4041)

# Event-Loop starten
if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(start_services())
